## Import

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

from IPython.display import Image

import seaborn as sns
from pylab import rcParams
# plt.style.use('fivethirtyeight')

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from imblearn.over_sampling import SMOTE

!pip install git+https://github.com/PyTorchLightning/pytorch-lightning
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from torch.utils.data import DataLoader, TensorDataset

import torch.nn as nn
import torchmetrics
import torch


import numpy as np

/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


  Cloning https://github.com/PyTorchLightning/pytorch-lightning to /tmp/pip-req-build-_ofz9dnw
  Running command git clone -q https://github.com/PyTorchLightning/pytorch-lightning /tmp/pip-req-build-_ofz9dnw
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 282 kB 5.1 MB/s 
     |████████████████████████████████| 829 kB 39.1 MB/s 
     |████████████████████████████████| 119 kB 44.2 MB/s 
     |████████████████████████████████| 636 kB 31.3 MB/s 
     |████████████████████████████████| 1.3 MB 39.7 MB/s 
     |████████████████████████████████| 142 kB 51.2 MB/s 
     |████████████████████████████████| 294 kB 47.2 MB/s 
  Created wheel for pytorch-lightning: filename=pytorch_lightning-1.5.0.dev0-py3-none-any.whl size=937232 sha256=75a76dfc3521e6548c6a303c81e7accf2610bd97adaaf1e595c56f84004a2dd0
  Stored in directory:

### hyperparameters

In [ ]:
NUM_EXAMPLES=562 
NUM_CLASSES=1
LR=0.001
HIDDEN_SIZE=64
BATCH_SIZE = 128

## Data Load

- 0인 경우 정상데이터, 1인 경우 이상 데이터

In [ ]:
df_path = '/content/drive/MyDrive/Deeplearning/uci-secom.csv'
df = pd.read_csv(df_path)

In [ ]:
df.head(3)

,Time,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,Pass/Fail
0,2008-07-19 11:55:00,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,0.0162,-0.0034,0.9455,202.4396,0.0,7.9558,414.8710,10.0433,0.9680,192.3963,12.5190,1.4026,-5419.00,2916.50,-4043.75,751.00,0.8955,1.7730,3.0490,64.2333,2.0222,0.1632,3.5191,83.3971,9.5126,50.6170,64.2588,49.3830,66.3141,86.9555,...,0.78,0.1827,5.7349,0.3363,39.8842,3.2687,1.0297,1.0344,0.4385,0.1039,42.3877,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,533.8500,2.1113,8.95,0.3157,3.0624,0.1026,1.6765,14.9509,NaN,NaN,NaN,NaN,0.5005,0.0118,0.0035,2.3630,NaN,NaN,NaN,NaN,-1
1,2008-07-19 12:32:00,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,-0.0005,-0.0148,0.9627,200.5470,0.0,10.1548,414.7347,9.2599,0.9701,191.2872,12.4608,1.3825,-5441.50,2604.25,-3498.75,-1640.25,1.2973,2.0143,7.3900,68.4222,2.2667,0.2102,3.4171,84.9052,9.7997,50.6596,64.2828,49.3404,64.9193,87.5241,...,1.33,0.2829,7.1196,0.4989,53.1836,3.9139,1.7819,0.9634,0.1745,0.0375,18.1087,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,535.0164,2.4335,5.92,0.2653,2.0111,0.0772,1.1065,10.9003,0.0096,0.0201,0.0060,208.2045,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045,-1
2,2008-07-19 13:17:00,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,0.0041,0.0013,0.9615,202.0179,0.0,9.5157,416.7075,9.3144,0.9674,192.7035,12.5404,1.4123,-5447.75,2701.75,-4047.00,-1916.50,1.3122,2.0295,7.5788,67.1333,2.3333,0.1734,3.5986,84.7569,8.6590,50.1530,64.1114,49.8470,65.8389,84.7327,...,0.85,0.0857,7.1619,0.3752,23.0713,3.9306,1.1386,1.5021,0.3718,0.1233,24.7524,267.064,0.9032,1.1,0.6219,0.4122,0.2562,0.4119,68.8489,535.0245,2.0293,11.21,0.1882,4.0923,0.0640,2.0952,9.2721,0.0584,0.0484,0.0148,82.8602,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602,1


In [ ]:
# To avoid an error
df = df.rename({"Pass/Fail":'Pass_Fail'},axis=1)
df.drop(columns="Time", axis=1, inplace=True)

In [ ]:
# 오류를 피하기 위해 -1을 0으로 바꿈
df['Pass_Fail'] = df['Pass_Fail'].replace([-1],0)

In [ ]:
df['Pass_Fail']

0       0
1       0
2       1
3       0
4       0
       ..
1562    0
1563    0
1564    0
1565    0
1566    0
Name: Pass_Fail, Length: 1567, dtype: int64

### Delete Null

In [ ]:
# 0%, 50%
def null_values(df, threshold):
    
    nv=pd.concat([df.isnull().sum(), 100 * df.isnull().sum()/df.shape[0]],axis=1).rename(columns={0:'Missing_Records', 1:'Percentage'})
    return nv[nv.Percentage > threshold].sort_values('Missing_Records', ascending=False)

null_50 = null_values(df,50)

df = df.drop(axis=1, columns=null_50.index)

## 남아있는 0 - 49% 내의 결측치를 갖고 있는 columns 결측치 처리
df.fillna(method='ffill', inplace=True)
df.fillna(method='bfill', inplace=True)
df.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,582,583,584,585,586,587,588,589,Pass_Fail
0,3030.93,2564.00,2187.7333,1411.1265,1.3602,100.0,97.6133,0.1242,1.5005,0.0162,-0.0034,0.9455,202.4396,0.0,7.9558,414.8710,10.0433,0.9680,192.3963,12.5190,1.4026,-5419.00,2916.50,-4043.75,751.00,0.8955,1.7730,3.0490,64.2333,2.0222,0.1632,3.5191,83.3971,9.5126,50.6170,64.2588,49.3830,66.3141,86.9555,117.5132,...,395.570,75.752,0.4234,12.93,0.78,0.1827,5.7349,0.3363,39.8842,3.2687,1.0297,1.0344,0.4385,0.1039,42.3877,267.064,0.9032,1.1,0.6219,0.4122,0.2562,0.4119,68.8489,533.8500,2.1113,8.95,0.3157,3.0624,0.1026,1.6765,14.9509,0.5005,0.0118,0.0035,2.3630,0.0096,0.0201,0.0060,208.2045,0
1,3095.78,2465.14,2230.4222,1463.6606,0.8294,100.0,102.3433,0.1247,1.4966,-0.0005,-0.0148,0.9627,200.5470,0.0,10.1548,414.7347,9.2599,0.9701,191.2872,12.4608,1.3825,-5441.50,2604.25,-3498.75,-1640.25,1.2973,2.0143,7.3900,68.4222,2.2667,0.2102,3.4171,84.9052,9.7997,50.6596,64.2828,49.3404,64.9193,87.5241,118.1188,...,408.798,74.640,0.7193,16.00,1.33,0.2829,7.1196,0.4989,53.1836,3.9139,1.7819,0.9634,0.1745,0.0375,18.1087,267.064,0.9032,1.1,0.6219,0.4122,0.2562,0.4119,68.8489,535.0164,2.4335,5.92,0.2653,2.0111,0.0772,1.1065,10.9003,0.5019,0.0223,0.0055,4.4447,0.0096,0.0201,0.0060,208.2045,0
2,2932.61,2559.94,2186.4111,1698.0172,1.5102,100.0,95.4878,0.1241,1.4436,0.0041,0.0013,0.9615,202.0179,0.0,9.5157,416.7075,9.3144,0.9674,192.7035,12.5404,1.4123,-5447.75,2701.75,-4047.00,-1916.50,1.3122,2.0295,7.5788,67.1333,2.3333,0.1734,3.5986,84.7569,8.6590,50.1530,64.1114,49.8470,65.8389,84.7327,118.6128,...,411.136,74.654,0.1832,16.16,0.85,0.0857,7.1619,0.3752,23.0713,3.9306,1.1386,1.5021,0.3718,0.1233,24.7524,267.064,0.9032,1.1,0.6219,0.4122,0.2562,0.4119,68.8489,535.0245,2.0293,11.21,0.1882,4.0923,0.0640,2.0952,9.2721,0.4958,0.0157,0.0039,3.1745,0.0584,0.0484,0.0148,82.8602,1


## Split data

In [ ]:
X = df.drop('Pass_Fail',axis=1)
y = df['Pass_Fail']
print(X.shape)
print(y.shape)
y

(1567, 562)
(1567,)


0       0
1       0
2       1
3       0
4       0
       ..
1562    0
1563    0
1564    0
1565    0
1566    0
Name: Pass_Fail, Length: 1567, dtype: int64

In [ ]:
# the function train_test_split creates random data samples (default: 75-25%)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 827)

# gettiing the shapes
print("shape of X_train: ", X_train.shape)
print("shape of X_test: ", X_test.shape)
print("shape of y_train: ", y_train.shape)
print("shape of y_test: ", y_test.shape)

shape of X_train:  (1175, 562)
shape of X_test:  (392, 562)
shape of y_train:  (1175,)
shape of y_test:  (392,)


### scaler

In [ ]:
sc = StandardScaler()

# fitting independent data to the model
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)
print(len(X_train))
X_train

1175


array([[ 0.61743244, -0.15275899, -0.90894117, ...,  0.18512201,
         0.38465797, -0.52263441],
       [-1.03534913,  0.0381048 , -2.82280744, ..., -0.25558743,
        -0.2802657 , -0.36977215],
       [-1.05598157, -0.16542529,  2.1715508 , ..., -0.82059953,
        -0.73521347, -0.43001441],
       ...,
       [-0.88368019,  0.70320987, -0.78349028, ..., -0.87710074,
        -0.84020142, -0.69006766],
       [-0.28219668,  0.29304519, -0.6562145 , ..., -0.09738404,
        -0.21027373, -0.3830774 ],
       [-0.82506219,  0.4276557 , -2.10912044, ..., -0.76409832,
        -0.73521347, -0.81548305]])

### Upsampling

In [ ]:
oversample = SMOTE()
x_tr_resample, y_tr_resample = oversample.fit_resample(X_train, y_train)
print(len(x_tr_resample))

2194


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
def count_suc_fail(ylist):
    success = 0
    fail = 0
    for i in ylist:
        if i == 0:
            success += 1
        elif i == 1:
            fail += 1

    print('success: ',success)
    print('fail: ', fail)


count_suc_fail(y_test)

success:  366
fail:  26


In [ ]:
count_suc_fail(y_tr_resample)

success:  1097
fail:  1097


In [ ]:
y_tr_resample

array([0, 0, 0, ..., 1, 1, 1])

In [ ]:
x_tr_resample.shape

(2194, 562)

## Extract test_sample

In [ ]:
test_sample = torch.Tensor(X_test[0])
test_sample.shape

torch.Size([562])

## Dataloadify

In [ ]:
tr_len = int(len(x_tr_resample) * 0.8)

x_tr = x_tr_resample[:tr_len]
y_tr = y_tr_resample[:tr_len]

x_val = x_tr_resample[tr_len:]
y_val = y_tr_resample[tr_len:]

tr_ds = TensorDataset(torch.Tensor(x_tr), torch.Tensor(y_tr))
val_ds = TensorDataset(torch.Tensor(x_val), torch.Tensor(y_val))
test_ds = TensorDataset(torch.Tensor(np.array(X_test)), torch.Tensor(np.array(y_test)))

tr_dl = DataLoader(tr_ds,batch_size=BATCH_SIZE,shuffle=True)
val_dl = DataLoader(val_ds,batch_size=BATCH_SIZE,shuffle=False)
test_dl = DataLoader(test_ds,batch_size=1,shuffle=False)

## Model

In [ ]:

class CNN(pl.LightningModule): #pytorch에서 LightningModule
    def __init__(self, num_examples, num_class, learning_rate, hidden_size): #self는 바구니
        super(CNN, self).__init__()

        self.num_examples = num_examples
        self.hid_size = hidden_size
        self.num_class = num_class
        self.linear = nn.Linear(self.num_examples, 128)

        self.cnn1 = torch.nn.Conv1d(
            in_channels=128,
            out_channels=self.hid_size,
            kernel_size=3,
            stride=1,
            padding=2)
        
        self.maxpool = torch.nn.MaxPool1d(kernel_size= 2, stride= 2)

        self.drop_out = torch.nn.Dropout(p=0.3)
        self.bn = torch.nn.BatchNorm1d(64)
        
        self.relu = torch.nn.ReLU()

        self.fc = torch.nn.Linear(64,self.num_class)
        self.sigmoid = torch.nn.Sigmoid()


        self.lr = learning_rate

        self.train_acc = torchmetrics.Accuracy()
        self.val_acc = torchmetrics.Accuracy()
        self.test_acc = torchmetrics.Accuracy()

    def forward(self, x):
        
        x = self.linear(x)
        
        x = self.relu(x)
        x = self.drop_out(x)
        x = self.cnn1(x.unsqueeze(2)) #억지로 차원을 늘려서 CNN에 맞춰주기
        x = self.maxpool(x)        
        x = self.relu(x)
        x = self.bn(x)
        x = x.squeeze(dim=2) #planting?
        x = self.fc(x)
        x = self.drop_out(x)
        x = self.sigmoid(x)

        return x

    @staticmethod
    def cross_entropy_loss(logits, labels):
        loss = nn.BCELoss()
        return loss(logits, labels.unsqueeze(dim=1))

    def training_step(self, batch, batch_nb):
        x, y = batch[0], batch[1]
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)

        self.train_acc(logits.round().squeeze(dim=1), y.type(torch.IntTensor))



        self.log("train/loss", loss, on_epoch=True)
        self.log("train/acc", self.train_acc, on_epoch=True)


        return loss

    def validation_step(self, batch, batch_nb):
        x, y = batch[0], batch[1]
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)

        self.val_acc(logits.round().squeeze(dim=1), y.type(torch.IntTensor))

        self.log("val/loss", loss, on_epoch=True)
        self.log("val/acc", self.val_acc, on_epoch=True)


    def test_step(self, batch, batch_nb):
        x, y = batch[0], batch[1]
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)

        self.test_acc(logits.round().squeeze(dim=1), y.type(torch.IntTensor))

        self.log("test/loss", loss, on_epoch=True)
        self.log("test/acc", self.test_acc, on_epoch=True)

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.lr, weight_decay=1e-5)

In [ ]:
cnn_model = CNN(num_examples=NUM_EXAMPLES,
                num_class=NUM_CLASSES,
                learning_rate=LR,
                hidden_size=HIDDEN_SIZE)

### set config to train model

In [36]:
checkpoint_callback = ModelCheckpoint(
    monitor="val/acc",
    dirpath="lightning_logs/",
    filename="Semicon_Project",  # +f'-{args.mode}',
    save_top_k=3,
    mode="max",
)

es = EarlyStopping(monitor="val/acc", mode="max", patience=5, verbose=True)

trainer = pl.Trainer(
    max_epochs=20,
    gpus=0,
    progress_bar_refresh_rate=20,
    auto_scale_batch_size=True,
    callbacks=[checkpoint_callback, es],
)

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:447: UserWarning: Checkpoint directory lightning_logs/ exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
GPU available: True, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:1367: UserWarning: GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`.
  "GPU available but not used. Set the gpus flag in your trainer `Trainer(gpus=1)` or script `--gpus=1`."


## Trainig and Test

In [37]:
trainer.fit(cnn_model,tr_dl,val_dl)
#trainer.test(test_dl)


   | Name      | Type        | Params
-------------------------------------------
0  | linear    | Linear      | 72.1 K
1  | cnn1      | Conv1d      | 24.6 K
2  | maxpool   | MaxPool1d   | 0     
3  | drop_out  | Dropout     | 0     
4  | bn        | BatchNorm1d | 128   
5  | relu      | ReLU        | 0     
6  | fc        | Linear      | 65    
7  | sigmoid   | Sigmoid     | 0     
8  | train_acc | Accuracy    | 0     
9  | val_acc   | Accuracy    | 0     
10 | test_acc  | Accuracy    | 0     
-------------------------------------------
96.9 K    Trainable params
0         Non-trainable params
96.9 K    Total params
0.388     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/data_loading.py:368: UserWarning: The number of training samples (14) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging interval"


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Monitored metric val/acc did not improve in the last 5 records. Best score: 1.000. Signaling Trainer to stop.


## Load Model

In [ ]:
loaded_model = CNN.load_from_checkpoint('/content/lightning_logs/7-trained-epoch=18-val_loss=0.00.ckpt',
                                    num_examples=NUM_EXAMPLES,
                                    num_class=NUM_CLASSES,
                                    learning_rate=LR,
                                    hidden_size=HIDDEN_SIZE)

RuntimeError: ignored

In [ ]:
def predict_fn(sample):

    score = loaded_model(sample)
    if score >= 0.5:
        return '이상치입니다.'
    else:
        return '정상입니다.'

## Run

In [ ]:
cnn_sample = test_sample.unsqueeze(0)

predict_fn(cnn_sample)